In [2]:
#Bring in modules
from bs4 import BeautifulSoup as bs
from splinter import Browser
import requests
import pymongo
import pandas as pd

In [3]:
#Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [4]:
#Define the database as mars and collection as items(for each article)
db = client.mars_db
collection = db.articles

In [34]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')

In [17]:

# Retrieve the parent divs for all the articles
results = soup.find_all(class_='list_text')
news_titles = []
articles = []

# Loop through returned results
for result in results:
        
        # Identify and return title of a news item
        title_div= result.find(class_='content_title')
        news_title = title_div.find('a').text
        news_titles.append(news_title)
        
        # Identify and return paragraph of the news item
        news_paragraph = result.find(class_='article_teaser_body').text
        articles.append(news_paragraph)

["Things Are Stacking up for NASA's Mars 2020 Spacecraft", "Curiosity Tastes First Sample in 'Clay-Bearing Unit'", "More Testing for Mars InSight's 'Mole'", 'NASA Garners 7 Webby Award Nominations', 'Curiosity Captured Two Solar Eclipses on Mars', "NASA's Mars Helicopter Completes Flight Tests", 'Asteroids, Hydrogen Make Great Recipe for Life on Mars', "Catch NASA's JPL at the Clippers SciFest This Weekend", "NASA's Mars 2020 Rover Is Put to the Test", "Opportunity's Parting Shot Was a Beautiful Panorama", 'Curiosity Resumes Operations After Switching Computers', 'InSight Lands Praise and a Proclamation from LA County', 'InSight Is the Newest Mars Weather Service', "Six Things to Know About NASA's Opportunity Rover", "NASA's Opportunity Rover Mission on Mars Comes to End", "NASA's InSight Prepares to Take Mars' Temperature", 'NASA to Share Results of Effort to Recover Mars Rover', "NASA's MAVEN Spacecraft Shrinking its Mars Orbit to Prepare for Mars 2020 Rover", '360 Video: Curiosity R

In [18]:
#executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
#browser = Browser('chrome', **executable_path, headless=False)

# Scrape and store Mars images
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)
# Retrieve page with the requests module
html = browser.html
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(html, 'html.parser')

In [20]:
# Capture Feature area box image url
mars = soup.find_all(class_="carousel_container")
for m in mars:
    image_url = m.find('a')['data-fancybox-href']
featured_image_url = 'https://www.jpl.nasa.gov'+image_url
#print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA16105_ip.jpg


In [22]:
# Scrape and store Mars Weather
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)
# Retrieve page with the requests module
html = browser.html
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(html, 'html.parser')
#Find and store mars_weather
mars_weather = soup.find(class_ = 'TweetTextSize TweetTextSize--normal js-tweet-text tweet-text').text
print(mars_weather)


InSight sol 141 (2019-04-20) low -98.3ºC (-144.9ºF) high -19.7ºC (-3.5ºF)
winds from the SW at 4.7 m/s (10.6 mph) gusting to 12.9 m/s (28.8 mph)
pressure at 7.40 hPapic.twitter.com/CQr1QQt3cM


In [23]:
#Scrape Mars Facts 
url = 'https://space-facts.com/mars/'
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')
#Scrape table and put into a panda dataframe
table = soup.find(class_="tablepress tablepress-id-mars")
table_rows = table.find_all('tr')
headers=[]
facts = []

#Use a loop to find content of each row
for tr in table_rows:
    header = tr.find('td', class_="column-1").text
    headers.append(header)
    fact = tr.find('td', class_="column-2").text
    facts.append(fact)
   
mars_facts=pd.DataFrame({"header":headers, "fact": facts})

#Remove non-data elements
mars_facts['fact']=mars_facts['fact'].str.replace('\n','')
mars_facts.head(10)

,header,fact
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [46]:
#Scrape images from Mars Hemispheres
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')
titles = []
image_urls=[]
web_locs=[]
#Retrieve the hemisphere names
hemispheres = soup.find_all(class_='description')

for h in hemispheres:
    #find web address of high res image
    web_loc = h.find('a',class_="itemLink product-item" )['href']
    web_locs.append(web_loc)
    image_url = 'https://astrogeology.usgs.gov' + web_loc
    image_urls.append(image_url)
    
    #find name of hemisphere   
    title = h.find('h3').text
    titles.append(title)

https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced
Cerberus Hemisphere Enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced
Schiaparelli Hemisphere Enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced
Syrtis Major Hemisphere Enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced
Valles Marineris Hemisphere Enhanced


In [47]:
#Go to each individual page to pull high res image
full_images=[]
for image_url in image_urls:
    browser.visit(image_url)
    
    # Retrieve page with the requests module
    html = browser.html
    soup = bs(html, 'html.parser')
    full= soup.find(class_='downloads')
    full_image = full.find('a')['href']
    full_images.append(full_image)

<div class="downloads">
<img class="thumb" src="/cache/images/dfaf3849e74bf973b59eb50dab52b583_cerberus_enhanced.tif_thumb.png"/>
<h3>Download</h3>
<ul>
<li><a href="http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg" target="_blank">Sample</a> (jpg) 1024px wide</li>
<li><a href="http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif" target="_blank">Original</a> (tif<span class="tooltip word-tif" title=""></span>) 21 MB</li>
</ul>
</div>
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
<div class="downloads">
<img class="thumb" src="/cache/images/7677c0a006b83871b5a2f66985ab5857_schiaparelli_enhanced.tif_thumb.png"/>
<h3>Download</h3>
<ul>
<li><a href="http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg" target="_blank">Sample</a> (jpg) 1024px wide</li>
<li><a href="http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schia

In [48]:
hemisphere_image_urls =pd.DataFrame({'title':titles, 
                                     'img_url':full_images})
hemisphere_image_urls.head()

,title,img_url
0,Cerberus Hemisphere Enhanced,http://astropedia.astrogeology.usgs.gov/downlo...
1,Schiaparelli Hemisphere Enhanced,http://astropedia.astrogeology.usgs.gov/downlo...
2,Syrtis Major Hemisphere Enhanced,http://astropedia.astrogeology.usgs.gov/downlo...
3,Valles Marineris Hemisphere Enhanced,http://astropedia.astrogeology.usgs.gov/downlo...
